In [1]:
import mysql.connector

# Параметры подключения
db = mysql.connector.connect(
    host="localhost",        # Хост базы данных
    user="root",    # Ваше имя пользователя
    password="Dorogusha1",# Ваш пароль
    database="zadanie2" # Имя базы данных
)

# Проверка подключения
if db.is_connected():
    print("Успешное подключение к базе данных!")


Успешное подключение к базе данных!


In [20]:
import mysql.connector

# Параметры подключения
db = mysql.connector.connect(
    host="localhost",        # Хост базы данных
    user="root",             # Ваше имя пользователя
    password="Dorogusha1",   # Ваш пароль
    database="zadanie2"      # Имя базы данных
)

cursor = db.cursor()

try:
    # Создание таблиц (включая поле 'modality' в таблице images и таблицу для статистики)
    create_tables_queries = [
        """
        CREATE TABLE IF NOT EXISTS patients (
            patient_id INT PRIMARY KEY,
            patient_name VARCHAR(255) NOT NULL,
            birth_date DATE,
            sex VARCHAR(10)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS images (
            id INT AUTO_INCREMENT PRIMARY KEY,
            file_path VARCHAR(255) NOT NULL,
            file_name VARCHAR(255) NOT NULL,
            resolution VARCHAR(50),
            modality VARCHAR(50),
            acquisition_date DATE,
            patient_id INT,
            upload_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (patient_id) REFERENCES patients(patient_id) ON DELETE CASCADE
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS image_processing (
            id INT AUTO_INCREMENT PRIMARY KEY,
            image_id INT,
            normalization VARCHAR(3),
            cropped_region VARCHAR(50),
            mask_applied VARCHAR(3),
            histogram_equalization VARCHAR(3),
            FOREIGN KEY (image_id) REFERENCES images(id) ON DELETE CASCADE
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS scan_type (
            modality VARCHAR(50) PRIMARY KEY,
            description TEXT
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS image_statistics (
            image_id INT PRIMARY KEY,
            mean_pixel_value FLOAT,
            std_pixel_value FLOAT,
            min_pixel_value FLOAT,
            max_pixel_value FLOAT,
            median_pixel_value FLOAT,
            entropy FLOAT,
            FOREIGN KEY (image_id) REFERENCES images(id) ON DELETE CASCADE
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS texture_characteristics1 (
            image_id INT PRIMARY KEY,
            contrast FLOAT,
            homogeneity FLOAT,
            correlation FLOAT,
            energy FLOAT,
            asm FLOAT,
            fractal_dimension FLOAT,
            lacunarity FLOAT,
            predicted_label VARCHAR(50),
            confidence_score FLOAT,
            segmentation_model_used VARCHAR(255),
            classification_model_used VARCHAR(255),
            processing_time FLOAT,
            FOREIGN KEY (image_id) REFERENCES images(id) ON DELETE CASCADE
        );
        """
    ]
    
    for query in create_tables_queries:
        cursor.execute(query)
        print("Таблица успешно создана.")

    # Проверка наличия столбца 'modality' в таблице 'images'
    cursor.execute("DESCRIBE images;")
    columns = [column[0] for column in cursor.fetchall()]

    if 'modality' not in columns:
        alter_table_query = """
        ALTER TABLE images ADD COLUMN modality VARCHAR(50);
        """
        cursor.execute(alter_table_query)
        print("Поле 'modality' успешно добавлено в таблицу images.")
    else:
        print("Столбец 'modality' уже существует в таблице images.")

    # Вставка данных пациента
    patient_data = (12345, "Иван Иванов", "1980-05-15", "Мужской")

    # Проверка на существование пациента с таким patient_id
    check_patient_query = "SELECT COUNT(*) FROM patients WHERE patient_id = %s"
    cursor.execute(check_patient_query, (patient_data[0],))
    patient_exists = cursor.fetchone()[0]

    if patient_exists == 0:
        insert_patient_query = """
        INSERT INTO patients (patient_id, patient_name, birth_date, sex)
        VALUES (%s, %s, %s, %s);
        """
        cursor.execute(insert_patient_query, patient_data)
        print(f"Данные пациента успешно добавлены: {patient_data}")
    else:
        print(f"Пациент с ID {patient_data[0]} уже существует.")

    # Вставка данных о снимке
    image_data = ('path/to/image1', 'image1.png', '256x256', 'MRI', '2024-01-01', 12345)
    insert_image_query = """
    INSERT INTO images (file_path, file_name, resolution, modality, acquisition_date, patient_id)
    VALUES (%s, %s, %s, %s, %s, %s);
    """
    cursor.execute(insert_image_query, image_data)
    print(f"Данные изображения успешно добавлены: {image_data}")

    # Вставка данных о предобработке изображения
    processing_data = (1, 'yes', 'hippocampus', 'yes', 'yes')  # Пример данных для обработки изображения
    insert_processing_query = """
    INSERT INTO image_processing (image_id, normalization, cropped_region, mask_applied, histogram_equalization)
    VALUES (%s, %s, %s, %s, %s);
    """
    cursor.execute(insert_processing_query, processing_data)
    print(f"Данные обработки изображения успешно добавлены: {processing_data}")

    # Вставка данных о типе снимка
    scan_type_data = ('MRI', 'Магнитно-резонансная томография')
    insert_scan_type_query = """
    INSERT INTO scan_type (modality, description)
    VALUES (%s, %s)
    ON DUPLICATE KEY UPDATE description = VALUES(description);
    """
    cursor.execute(insert_scan_type_query, scan_type_data)
    print(f"Данные типа снимка успешно добавлены или обновлены: {scan_type_data}")

    # Вставка данных о статистике изображения (среднее, стандартное отклонение и другие метрики)
    statistics_data = (
        1,                 # image_id
        150.0,             # mean_pixel_value
        20.5,              # std_pixel_value
        50.0,              # min_pixel_value
        200.0,             # max_pixel_value
        140.0,             # median_pixel_value
        5.2                # entropy
    )  
    insert_statistics_query = """
    INSERT INTO image_statistics (
        image_id, 
        mean_pixel_value, 
        std_pixel_value, 
        min_pixel_value, 
        max_pixel_value, 
        median_pixel_value, 
        entropy
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        mean_pixel_value = VALUES(mean_pixel_value),
        std_pixel_value = VALUES(std_pixel_value),
        min_pixel_value = VALUES(min_pixel_value),
        max_pixel_value = VALUES(max_pixel_value),
        median_pixel_value = VALUES(median_pixel_value),
        entropy = VALUES(entropy);
    """
    cursor.execute(insert_statistics_query, statistics_data)
    print(f"Статистика изображения успешно добавлена или обновлена: {statistics_data}")

    # Вставка текстурных характеристик
    texture_data = (
        1,                 # image_id
        0.8,               # contrast
        0.85,              # homogeneity
        0.75,              # correlation
        0.6,               # energy
        0.7,               # asm (Angular Second Moment)
        1.2,               # fractal_dimension
        0.95,              # lacunarity
        'autism',          # predicted_label
        0.85,              # confidence_score
        'U-Net',           # segmentation_model_used
        'SVM',             # classification_model_used
        12.5               # processing_time
    )
    insert_texture_query = """
    INSERT INTO texture_characteristics1 (
        image_id, 
        contrast, 
        homogeneity, 
        correlation, 
        energy, 
        asm, 
        fractal_dimension, 
        lacunarity, 
        predicted_label, 
        confidence_score, 
        segmentation_model_used, 
        classification_model_used, 
        processing_time
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        contrast = VALUES(contrast),
        homogeneity = VALUES(homogeneity),
        correlation = VALUES(correlation),
        energy = VALUES(energy),
        asm = VALUES(asm),
        fractal_dimension = VALUES(fractal_dimension),
        lacunarity = VALUES(lacunarity),
        predicted_label = VALUES(predicted_label),
        confidence_score = VALUES(confidence_score),
        segmentation_model_used = VALUES(segmentation_model_used),
        classification_model_used = VALUES(classification_model_used),
        processing_time = VALUES(processing_time);
    """
    cursor.execute(insert_texture_query, texture_data)
    print(f"Текстурные характеристики успешно добавлены или обновлены: {texture_data}")

    # Сохранение изменений в базе данных
    db.commit()

except mysql.connector.Error as err:
    print(f"Ошибка при выполнении запроса: {err}")
    db.rollback()

finally:
    # Закрытие соединения с базой данных
    cursor.close()
    db.close()


Таблица успешно создана.
Таблица успешно создана.
Таблица успешно создана.
Таблица успешно создана.
Таблица успешно создана.
Таблица успешно создана.
Столбец 'modality' уже существует в таблице images.
Пациент с ID 12345 уже существует.
Данные изображения успешно добавлены: ('path/to/image1', 'image1.png', '256x256', 'MRI', '2024-01-01', 12345)
Данные обработки изображения успешно добавлены: (1, 'yes', 'hippocampus', 'yes', 'yes')
Данные типа снимка успешно добавлены или обновлены: ('MRI', 'Магнитно-резонансная томография')
Статистика изображения успешно добавлена или обновлена: (1, 150.0, 20.5, 50.0, 200.0, 140.0, 5.2)
Текстурные характеристики успешно добавлены или обновлены: (1, 0.8, 0.85, 0.75, 0.6, 0.7, 1.2, 0.95, 'autism', 0.85, 'U-Net', 'SVM', 12.5)


In [21]:
###### import mysql.connector
import pandas as pd
import os

# Параметры подключения
db = mysql.connector.connect(
    host="localhost",        # Хост базы данных
    user="root",             # Ваше имя пользователя
    password="Dorogusha1",   # Ваш пароль
    database="zadanie2"      # Имя базы данных
)

cursor = db.cursor()

# Извлечение списка всех таблиц в базе данных
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()

# Создание пути для сохранения файла Excel
output_file = "database_data_all_tables6.xlsx"

# Сохранение всех таблиц в один файл Excel
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for table in tables:
        table_name = table[0]
        # Извлечение данных из текущей таблицы
        cursor.execute(f"SELECT * FROM {table_name};")
        table_data = cursor.fetchall()

        # Получение названий столбцов для текущей таблицы
        columns = [column[0] for column in cursor.description]

        # Преобразование данных в DataFrame
        df = pd.DataFrame(table_data, columns=columns)

        # Запись данных в файл Excel в новый лист
        df.to_excel(writer, sheet_name=table_name, index=False)

print(f"Данные сохранены в файл '{output_file}' с несколькими листами.")

# Закрытие соединения с базой данных
cursor.close()
db.close()


Данные сохранены в файл 'database_data_all_tables6.xlsx' с несколькими листами.
